In [2]:
# Importing libs
from huggingface_hub import hf_hub_download
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
from trl import SFTTrainer, SFTConfig


In [ ]:
# Loading and cofiguring model
model_name= 'deepseek-ai/deepseek-coder-6.7b-instruct'
device='auto'

model=AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
).to(device)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token - tokenizer.eos_token

peft_config = LoraConfig(
    r=16,  # Rank (can go up to 64 for quality)
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"]  # Which layers to adapt
)

model =get_peft_model(model, peft_config)
model.print_trainable_parameters()

In [5]:
# ============================================================
# PREPROCESSING FOR MAGICODER-EVOL-INSTRUCT-110K
# ============================================================
from datasets import load_dataset
# Load dataset and split 
dataset = load_dataset("ise-uiuc/Magicoder-Evol-Instruct-110K", split="train")
dataset = dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = dataset['train']  # ~99K
val_dataset = dataset['test']     # ~11K
print(f"Train: {len(train_dataset)} | Val: {len(val_dataset)}")


# 3. Preprocessing function
def preprocess_dataset(examples, tokenizer):
    """
    Format Magicoder examples with chat template.
    
    Magicoder structure:
    {
        'instruction': 'Write a function to...',
        'response': 'def foo():\n    ...'
    }
    """

    texts = []
    for instruction, response in zip(examples["instruction"], examples["response"]):
        messages = [
            {"role": "system", "content": "You are an expert Python programmer specializing in clean, efficient code."},
            {"role": "user", "content": instruction},
            {"role": "assistant", "content": response}
        ]
        
        # Apply chat template (handles special tokens automatically)
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False
        )
        texts.append(text)
    
    return {"text": texts}
    
# 4. Apply preprocessing (PARALLELIZED with 64 CPUs!)
train_formatted = train_dataset.map(
    lambda x: preprocess_dataset(x, tokenizer),
    batched=True,
    batch_size=1000,
    num_proc=4,      # Use all 64 vCPUs!
    remove_columns=train_dataset.column_names,
    desc="Formatting train"
)
val_formatted = val_dataset.map(
    lambda x: preprocess_dataset(x, tokenizer),
    batched=True,
    batch_size=1000,
    num_proc=4,
    remove_columns=val_dataset.column_names,
    desc="Formatting val"
)
print(f"✅ {len(train_data)}Preprocessing complete!")

Train: 100064 | Val: 11119


Formatting train (num_proc=4):   0%|          | 0/100064 [00:00<?, ? examples/s]

NameError: name 'preprocess_dataset' is not defined

In [ ]:

# Training with validation
output_dir="sft_results"

train_args=SFTConfig(
    output_dir=output,
    learning_rate=3e-5, #3e-5
    warmup_ratio=0.1,

    # gradient_checkpointing=True,
    weight_decay=0.1,
    max_grad_norm=0.3,
    lr_scheduler_type="cosine",
    bf16=True,
    optim="paged_adamw_8bit",

    # Epochs, Batches and saving
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,  # 2x training
    gradient_accumulation_steps=1,
    evaluation_strategy="steps",
    eval_steps=500,
    
    save_strategy="steps",
    save_steps=1000,
    save_total_limit=1,  # Keep only 3 best checkpoints

    # Logging
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    report_to="none",

    max_seq_length=1024,
    packing=False,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,  # ← Validation here
    args=train_args
)

In [ ]:
# Train and auto-evaluate
trainer.train()
trainer.save_model(output_dir)# Final evaluation

eval_results = trainer.evaluate()
print(f"Final validation loss: {eval_results['eval_loss']:.4f}")